In [3]:
import os
os.environ['DDE_BACKEND'] = 'tensorflow.compat.v1'
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import deepxde as dde
import tensorflow as tf
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator

# Geometry parameters
L = 6e-2  # m
R = 4e-3  # m
t_last = 1e6  # s

# Geometry definition
square = dde.geometry.Rectangle([0, 0], [L, L])
circle = dde.geometry.Disk([0, 0], R)
geom = dde.geometry.csg.CSGDifference(square, circle)
timedomain = dde.geometry.TimeDomain(0, t_last)
geomtime = dde.geometry.GeometryXTime(geom, timedomain)

# Load and prepare data for sigma_H model
df = pd.read_excel('Hydrostatic Data with Nodes.xlsx')
n = 400
sampled_data = df.sample(n=n)
train_data, test_data = train_test_split(sampled_data, test_size=0.4, random_state=42)
x_train = np.array(train_data['x'].values).reshape(-1, 1)
y_train = np.array(train_data['y'].values).reshape(-1, 1)
sigma_train = np.array(train_data['sigma'].values).reshape(-1, 1)
x_test = np.array(test_data['x'].values).reshape(-1, 1)
y_test = np.array(test_data['y'].values).reshape(-1, 1)
sigma_test = np.array(test_data['sigma'].values).reshape(-1, 1)
data_x = np.column_stack((x_train, y_train))
data_sigma = sigma_train
test_x = np.column_stack((x_test, y_test))
test_sigma = sigma_test
data = dde.data.dataset.DataSet(X_train=data_x, y_train=data_sigma, X_test=test_x, y_test=test_sigma)
layer_size = [2] + [50] * 5 + [1]
activation = "relu"
initializer = "He normal"
net_sigma = dde.nn.FNN(layer_size, activation, initializer)
model_sigma = dde.Model(data, net_sigma)
model_sigma.compile("adam", lr=1e-5)
losshistory, train_state = model_sigma.train(epochs=100000)

# Create spatial grid for sigma_H and its derivatives
Nx, Ny = 100, 100  # Grid resolution
x_grid = np.linspace(0, L, Nx)
y_grid = np.linspace(0, L, Ny)
X_grid, Y_grid = np.meshgrid(x_grid, y_grid)
grid_points = np.column_stack([X_grid.ravel(), Y_grid.ravel()])

# Predict sigma_H on the grid
sigma_H_values = model_sigma.predict(grid_points).reshape(Nx, Ny)

# Compute numerical derivatives
sigma_H_x = np.zeros_like(sigma_H_values)
sigma_H_y = np.zeros_like(sigma_H_values)
sigma_H_xx = np.zeros_like(sigma_H_values)
sigma_H_yy = np.zeros_like(sigma_H_values)

dx = L / (Nx - 1)
dy = L / (Ny - 1)

for i in range(1, Nx - 1):
    for j in range(1, Ny - 1):
        sigma_H_x[i, j] = (sigma_H_values[i + 1, j] - sigma_H_values[i - 1, j]) / (2 * dx)
        sigma_H_y[i, j] = (sigma_H_values[i, j + 1] - sigma_H_values[i, j - 1]) / (2 * dy)
        sigma_H_xx[i, j] = (sigma_H_values[i + 1, j] - 2 * sigma_H_values[i, j] + sigma_H_values[i - 1, j]) / dx**2
        sigma_H_yy[i, j] = (sigma_H_values[i, j + 1] - 2 * sigma_H_values[i, j] + sigma_H_values[i, j - 1]) / dy**2

# Handle boundaries (example: zero derivatives)
sigma_H_x[0, :] = sigma_H_x[-1, :] = 0
sigma_H_y[:, 0] = sigma_H_y[:, -1] = 0
sigma_H_xx[0, :] = sigma_H_xx[-1, :] = 0
sigma_H_yy[:, 0] = sigma_H_yy[:, -1] = 0

# Create interpolators
sigma_H_interp = RegularGridInterpolator((x_grid, y_grid), sigma_H_values)
sigma_H_x_interp = RegularGridInterpolator((x_grid, y_grid), sigma_H_x)
sigma_H_y_interp = RegularGridInterpolator((x_grid, y_grid), sigma_H_y)
sigma_H_xx_interp = RegularGridInterpolator((x_grid, y_grid), sigma_H_xx)
sigma_H_yy_interp = RegularGridInterpolator((x_grid, y_grid), sigma_H_yy)

# Define initial condition
def initial_condition(x):
    return 20.0  # Example initial concentration

ic = dde.icbc.IC(geomtime, initial_condition, lambda x, on_initial: on_initial, component=0)

# Define PDE with numerical derivatives
def pde_C_L(x, y):
    D_L = 3.8e-11  # m^2/s
    V_H = 2e-6     # m^3/mole
    RT = 8.314 * 298  # J/mole

    Coe = (D_L * V_H) / RT
    C_L = y[:, 0:1]

    # Compute derivatives of C_L using automatic differentiation
    dC_dt = dde.grad.jacobian(y, x, i=0, j=2)
    CL_x = dde.grad.jacobian(y, x, i=0, j=0)
    CL_y = dde.grad.jacobian(y, x, i=0, j=1)
    CL_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
    CL_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)

    # Evaluate sigma_H and its derivatives using interpolators
    sigma_H_vals = sigma_H_interp(x[:, :2])
    sigma_H_x_vals = sigma_H_x_interp(x[:, :2])
    sigma_H_y_vals = sigma_H_y_interp(x[:, :2])
    sigma_H_xx_vals = sigma_H_xx_interp(x[:, :2])
    sigma_H_yy_vals = sigma_H_yy_interp(x[:, :2])

    # Reshape to match dimensions
    sigma_H_vals = sigma_H_vals.reshape(-1, 1)
    sigma_H_x_vals = sigma_H_x_vals.reshape(-1, 1)
    sigma_H_y_vals = sigma_H_y_vals.reshape(-1, 1)
    sigma_H_xx_vals = sigma_H_xx_vals.reshape(-1, 1)
    sigma_H_yy_vals = sigma_H_yy_vals.reshape(-1, 1)

    # Compute the PDE residual
    eq1 = (
        dC_dt
        - D_L * (CL_xx + CL_yy)
        + Coe * (
            CL_x * sigma_H_x_vals
            + CL_y * sigma_H_y_vals
            + C_L * (sigma_H_xx_vals + sigma_H_yy_vals)
        )
    )

    return [eq1]

# Set up the data for PDE problem
data_pde = dde.data.TimePDE(
    geomtime,
    pde_C_L, [ic],
    num_domain=500,
    num_boundary=500,
    num_initial=500,
    num_test=500,
)

# Define the neural network for C_L
layer_size = [3] + [64] * 5 + [1]  # 3 inputs (x, y, t) -> hidden layers -> 1 output (C_L)
activation = "tanh"
initializer = "Glorot normal"
net_C_L = dde.nn.FNN(layer_size, activation, initializer)

# Create and compile the model
model = dde.Model(data_pde, net_C_L)
model.compile("adam", lr=1e-4)
losshistory, train_state = model.train(epochs=100000, display_every=1000)

# Save and plot the results
dde.saveplot(losshistory, train_state, issave=True, isplot=True)


Compiling model...
Building feed-forward neural network...
'build' took 0.113781 s


'compile' took 0.420586 s

Training model...

Step      Train loss    Test loss     Test metric
0         [2.37e+15]    [2.33e+15]    []  
1000      [2.37e+15]    [2.33e+15]    []  
2000      [2.37e+15]    [2.33e+15]    []  
3000      [2.37e+15]    [2.33e+15]    []  
4000      [2.37e+15]    [2.33e+15]    []  
5000      [2.37e+15]    [2.33e+15]    []  
6000      [2.37e+15]    [2.33e+15]    []  
7000      [2.37e+15]    [2.33e+15]    []  
8000      [2.37e+15]    [2.33e+15]    []  
9000      [2.37e+15]    [2.33e+15]    []  
10000     [2.37e+15]    [2.33e+15]    []  
11000     [2.37e+15]    [2.33e+15]    []  
12000     [2.37e+15]    [2.33e+15]    []  
13000     [2.37e+15]    [2.33e+15]    []  
14000     [2.37e+15]    [2.33e+15]    []  
15000     [2.37e+15]    [2.33e+15]    []  
16000     [2.37e+15]    [2.33e+15]    []  
17000     [2.37e+15]    [2.33e+15]    []  
18000     [2.37e+15]    [2.33e+15]    []  
19

NotImplementedError: Cannot convert a symbolic tf.Tensor (strided_slice_8:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported.